In [3]:
import math
import nbimporter
import PsychrometricProperty as psy

In [4]:
def Tdp_ptW(p, T, W):
    """
    Determine the dew point temperature for air/water vapor.

    Parameters:
        p (float): Absolute pressure of the mixture [psia].
        T (float): Dry bulb temperature [F].
        r (float): Relative humidity [%].

    Returns:
        float: Dew point temperature [F].
    """
    # Initialize coefficients
    C = [0, 100.45, 33.193, 2.319, 0.17074, 1.2063]

    # Determine the partial pressure of water vapor in the air, [psia]
    pw = (p*W)/(0.621945 + W)
    alpha = math.log(pw)

    # Initial dew point temperature estimate
    Tdp_test = 90.12 + 26.142 * alpha + 0.8972 * alpha**2

    if Tdp_test < 32:
        T_dp =  Tdp_test
        return T_dp
    elif 32 <= Tdp_test <= 200:
        T_dp =  (C[1] + C[2] * alpha + C[3] * alpha**2 +
                C[4] * alpha**3 + C[5] * pw**0.1984)
        return T_dp
    else:
        raise ValueError("Dew point temperature out of valid range")

# Homework #3

### Problem 5.6. Mixing of two stream
A flow of 25,000L/s of outside air at 10C and 80% RH mixes with 100,000 L/s of return air at 24C and 45% RH.\
a. Determine the temperature, humidity ratio, and enthalpy of the supply (mixed) stream.\
b. It is desired to keep the total supply flow rate the same (125,000 L/s) but have an outdoor air flow fraction that is 15% of the supply flow. Determine the outdoor air requirement and the new supply state.\
c. Plot the processes on a psychometric plot and draw some conclusions from your results.

### Problem 5.9. Cooling coil
A flow of 4000cfm of air at 80F db and 66F wb flows through a cooling coil. The air leaves at 47F and 93% RH. Determine the rate of moisture removal (lbm/hr), the total energy required to cool the air stream (Btu/hr), and the latent and sensible components of the total energy required for cooling.

### Problem 5.16. Zone cooling load
The total cooling load for a building zone is 100 kW with a sensible load ratio of 0.75. The zone set point is 22C. The supply air is at 10C and 95% RH. Determine the supply air flow rate required to maintain the zone at the desired temperature, the resulting zone humidity ratio, and the sensible heat ratio of the zone. Plot the process on a psychrometric plot.

### Problem 5.16. Heating and Humidification
A flow stream of air at atmospheric pressure is to be heated and humidified from an inlet temperature of 18 C and relative humidity of 20% to a final temperature of 40C and relative humidity of 40%. The air is first heated by a hot water coil followed by the injection of saturated water vapor at a pressure 200kPa. The mass flow rate of air is 1.5 kg/s. Determine the heat transfer from the hot water coil to the air stream and the mass flow rate of injected water vapor necessary to obtain the desired exit conditions. Show how the process would look on a psychrometric chart. Draw some conclu- sions from your results.


### Problem 5.19
A 4000 ft3 room is maintained at 70 F, 50% relative humidity, and 14.7 psia. Mechanical ventilation is provided by a central system, while heating and humidification are performed locally at the zone. The ventilation air is provided at 40 F and 30% relative humidity with a flow rate of 50 cfm. There is a total heat loss from the space through the walls of 5000 Btu/hr. A humidifier adds moisture in the form of saturated vapor at 212 F. Assuming the room air is fully mixed, determine the following: 
a. the amount of moisture addition necessary.\
b. the amount of heating required\
c. The mass of water vapor within the room.\
d. the temperature of surfaces within the room at which moisture will condense. 
e. a plot of the process on psychrometric coordinates 
f. Draw some conclusions from your results. 


#### Solutions:

List all the given information:

In [5]:

P_atm = 14.7 # psia
V_z = 4000 # ft^3
T_z = 70 # space zone teperature, F
RH_z = 50 # spce zone humidity, %

T_SA = 40 # supply air temperature, F
RH_SA = 30 # supply air humidity, %
V_dot_SA = 50 # supply air volume flow rate

q_loss = 5000 # Btu/hr
T_s = 212 # saturated vapor temperature,F
h_s = 1150 # saturated vapor enthalpy,btu/hr

Determine the state for space zone and supply air

In [6]:
h_z = psy.h_ptr(P_atm,T_z,RH_z) # enthalpy of space air, btu/hr
w_z = psy.W_ptr(P_atm,T_z,RH_z) # humidity ratio of space air, lbm_w/lbm_a
v_z = psy.v_ptr(P_atm,T_z,RH_z) # specifiv volume of space air, ft^3/lb

h_SA = psy.h_ptr(P_atm,T_SA,RH_SA) # enthalpy of supply air, btu/hr
w_SA = psy.W_ptr(P_atm,T_SA,RH_SA) # humidity ratio of supply air, lbm_w/lbm_a
v_SA = psy.v_ptr(P_atm,T_SA,RH_SA) # specifiv volume of supply air, ft^3/lb

Calculate the mass of air in the space and mass flow rate of supply air

In [7]:
M_z = V_z /v_z # mass of air in space, lb/hr
m_dot_SA = V_dot_SA * 60/v_SA # mass flow rate of supply air, lb/hr

Based on the conservation of mass of mositure:
$$\dot{m}_{SA} w_{SA} + \dot{m}_s = \dot{m}_{SA} w_{z}$$
The amount of moisture is:
$$\dot{m}_s = \dot{m}_{SA} (w_{z}-w_{SA})$$
a) the amount of moisture required:

In [8]:
m_dot_s = m_dot_SA * (w_z-w_SA)
print('a) The amount of moisture is:', m_dot_s, 'lbm/hr')

a) The amount of moisture is: 1.4812584714955404 lbm/hr


Based on the conservation of energy :
$$\dot{m}_{SA} h_{SA} + \dot{m}_s h_s + \dot{q}_{heat} = \dot{m}_{SA} h_{z} + \dot{q}_{loss}$$
b) the amount of heating required is:
$$\dot{q}_{heat} = \dot{m}_{SA} h_{z} + \dot{q}_{loss}-\dot{m}_{SA} h_{SA} - \dot{m}_s h_s $$

In [9]:
q_dot_heat = m_dot_SA*h_z + q_loss - m_dot_SA*h_SA - m_dot_s*h_s
print('b) The amount of heating required is', q_dot_heat,'Btu/hr')

b) The amount of heating required is 6630.373251292586 Btu/hr


c) The mass of water vapor within the room.

In [10]:
M_w = M_z * w_z
print('c) The mass of water vapor within the room is', M_w, 'lbm')

c) The mass of water vapor within the room is 2.303159746192362 lbm


d) the temperature of surfaces within the room at which moisture will condense is the dewpoint temperature at the zone space temperature and humidity.

In [11]:
T_dp_z = psy.Tdp_ptr(P_atm,T_z,RH_z) #F
print('d)the temperature of surfaces within the room at which moisture will condense is', T_dp_z,'F')

d)the temperature of surfaces within the room at which moisture will condense is 50.59130226051858 F


### Problem 3

The cooling loads in a commercial building are being met with the help of a dedicated outdoor air system (DOAS) that operates in parallel with a primary cooling coil as shown in the attached schematic. The DOS first dehumidifies the outdoor air (which enters at 90°F dry-bulb, 75°F wet- bull) with the help of a solid desiccant wheel, which lowers the humidity ratio to a final value of 0.009 Ibw/Ibda- This ventilation air is then cooled against the exhaust air using a heat recovery ventilator with 80% sensible effectiveness. The ventilation air is then further cooled with an auxiliary cooling coil before being mixed with the primary air. The primary air is sensibly cooled to a dry-bulb temperature of 67°F. Return air with a mass flow rate of 3084 Ibm/hr is extracted from the space at 81°F dry-bulb and 71°F wet-bulb. Supply air to the space enters at 67°F dry-bulb temperature. The space has a total cooling load of 15,000 Btu/hr. Determine the following:\
a) The sensible heat ratio of the space; \
b) The required outdoor air flow rate (Ib/hr); \
c) The cooling capacity (Btu/hr) of the auxiliary cooling coil and the primary cooling coil; \
d) The drybulb temperature entering/leaving each device in the DOAS; \
e) Plot all processes on a psychrometric chart.

#### Solutions:

a)The sensible heat ratio of the space\
$$SHR = \frac{\dot{Q}_{sen}}{\dot{Q}_{total}}$$
The sensible heat the used to increase the temperature of air from supply state to the return state. Thus, we can create a point x, which the temperature is equal to the supply air temperature but the humidity ratio is equal to the return air humidity ratio.
$$h_x = h@(P_{atm}, T_{SA}, w_{RA})$$
The sensible heating load is:
$$\dot{Q}_{sen} = \dot{m}_{RA} (h_{RA}-h_x)$$


In [12]:
P_atm = 14.7
m_dot_RA = 3084 # mass flow rate of return air, lbm/hr
T_db_RA = 81 # return air dry bulb temperature,F
T_wb_RA = 71 # return air wet bulb temperature,F
T_db_SA = 67 # supply air dry bulb temperature,F
W_RA  = psy.W_ptb(P_atm,T_db_RA, T_wb_RA)
h_RA  = psy.h_ptb(P_atm,T_db_RA, T_wb_RA)
h_x = 0.24 * T_db_SA + W_RA * (1061+0.444*T_db_SA)
Q_dot_sen = m_dot_RA * (h_RA - h_x)
Q_dot_total = 15000 # total heating load, Btu/hr
SHR = Q_dot_sen/Q_dot_total
print ('sensible heat load is:',Q_dot_sen,'Btu/hr')
print ('SHR is:',SHR,'Btu/hr')

sensible heat load is: 10630.287265747973 Btu/hr
SHR is: 0.7086858177165315 Btu/hr


b) The psychrometric process of solid desiccant is a decrease in humidity while the dry bulb temperature increases slightly due to the heat released during the adsorption process. Ideally, it can be seen as an isenthalpic process (h is constant).

Calculate the enthapy of outdoor air

In [13]:
T_db_OA = 90 # outdoor air dry bulb temperature,F
T_wb_OA = 75 # outdoor air wet bulb temperature,F
W_OA = psy.W_ptb(P_atm,T_db_OA,T_wb_OA) # outdoor air humidity ratio, lbm_w\lbm_a
h_OA = psy.h_ptb(P_atm,T_db_OA,T_wb_OA) # outdoor air enthalpy,btu/lb

Determine the temperature flowing out of the solid desiccant

In [14]:
h_SD = h_OA # isenthalpic process in solid desiccant
W_SD = 0.009 # humidity ratio at the outlet of solid desiccant, lbmw/lbma
T_db_SD = (h_SD-1061*W_SD)/(0.24+0.444*W_SD) # temperature at the outlet of solid desiccant
print('The temperature at the outlet of solid desiccant is',T_db_SD,'F')

The temperature at the outlet of solid desiccant is 118.08275500316076 F


The definition of sensible effectiveness of an heat recovery ventilator (HRV) is:
$$\epsilon_{sen} = \frac{T_{SD}-T_{VA}}{T_{SD}-T_{RA}}$$
so the temperature of outdoor ait flowing out of the HRV:
$$T_{VA} = T_{SD} - \epsilon_{sen} (T_{SD}-T_{RA})$$
Based on the mass balance and energy balance in HRV:
$$\dot{m}_{OA} = \dot{m}_{EA}$$
$$\dot{m}_{OA} c_{p,OA} (T_{OA}-T_{VA}) = \dot{m}_{EA} c_{p,EA} (T_{EA}-T_{RA})$$
$c_{p,OA}$ is roughtly equal to $c_{p,EA}$. so:
$$T_{EA} = (T_{OA}-T_{VA}) + T_{RA}$$
In HRV, there is no mositure transfer, so the outdoor humidity ratio does not change.

In [15]:
epsilon_sen = 0.8 # HRV sensible effectiveness
T_db_VA = T_db_SD - epsilon_sen * (T_db_SD-T_db_RA)
W_VA = W_SD
h_VA = 0.24 * T_db_VA + W_VA * (1061+0.444*T_db_VA)
print('The temperature of outdoor air at the outlet of HRV is',T_db_VA,'F')

T_db_EA = (T_db_SD-T_db_VA) + T_db_RA
print('The temperature of exhaust air at the outlet of HRV is',T_db_EA,'F')

The temperature of outdoor air at the outlet of HRV is 88.41655100063215 F
The temperature of exhaust air at the outlet of HRV is 110.6662040025286 F


Draw a control volume include Aux. cooling coil, primary cooling coil and mixing box, as control volume 1 (CV1), and treat the space as control volume 2 (CV2).\
Based on the mass and energy balance in CV2:
$$\dot{m}_{SA} = \dot{m}_{RA} $$
$$\dot{Q}_{total} = \dot{m}_{RA} (h_{RA}-h_{SA})$$
to obtain $h_{SA}$

In [16]:
m_dot_SA = m_dot_RA
h_SA = h_RA - Q_dot_total/m_dot_RA

If we want to know whether the psychrometric process is sensible only cooling or cooling and dehumidification. So we calculate the dew point temperature at inlet of auxiliary $Tdp_{VA}$.

In [17]:
T_dp_VA = Tdp_ptW(P_atm,T_db_VA,W_VA)
T_dp_RA = psy.Tdp_ptb(P_atm,T_db_RA,T_wb_RA)
print('The dew point temperature of air at the outlet of aux cooling coil is',T_dp_VA,'F')
print('The dew point temperature of air at the outlet of primary cooling coil is',T_dp_RA,'F')



The dew point temperature of air at the outlet of aux cooling coil is 54.49066710491119 F
The dew point temperature of air at the outlet of primary cooling coil is 66.63718607603103 F


Thus, bsed on the dew point temperature at the outlet of aux. and primary cooling coil, we can know that only sensible cooling occurs in the auxiliry and primary cooling coil.

In [18]:
h_aux_out = 0.24 * T_db_SA + W_SD * (1061+0.444*T_db_SA)
h_pri_out = 0.24 * T_db_SA + W_RA * (1061+0.444*T_db_SA)
print('The enthalpy of air at the outlet of aux cooling coil is',h_aux_out,'Btu/lb')
print('The enthalpy of air at the outlet of primary cooling coil is',h_pri_out,'Btu/lb')

The enthalpy of air at the outlet of aux cooling coil is 25.896732 Btu/lb
The enthalpy of air at the outlet of primary cooling coil is 31.33142529028776 Btu/lb


Based on the conservation of mass and energy in CV1:
$$\dot{m}_{aux,out} + \dot{m}_{pri,out} = \dot{m}_{SA}$$
$$\dot{m}_{aux,out} h_{aux,out}+ \dot{m}_{pri,out} h_{pri,out} = \dot{m}_{SA} h_{SA}$$
so:
$$\dot{m}_{aux,out} = \frac{\dot{m}_{SA} h_{pri,out}-\dot{m}_{SA} h_{SA}}{h_{pri,out}-h_{aux,out}}$$
$$\dot{m}_{pri,out} = \frac{\dot{m}_{SA} h_{aux,out}-\dot{m}_{SA} h_{SA}}{h_{aux,out}-h_{pri,out}}$$

In [19]:
m_aux_out = (m_dot_SA*h_pri_out-m_dot_RA*h_SA)/(h_pri_out-h_aux_out)
m_pri_out = (m_dot_SA*h_aux_out-m_dot_RA*h_SA)/(h_aux_out-h_pri_out)

c) The cooling capacity of the auxiliary cooling coil and the primary cooling coil;
$$\dot{Q}_{aux,coil} = \dot{m}_{aux,out} (h_{VA}-h_{aux,out})$$
$$\dot{Q}_{pri,coil} = \dot{m}_{pri,out} (h_{RA}-h_{pri,out})$$

In [20]:
Q_aux_coil = m_aux_out*(h_VA-h_aux_out)
Q_pri_coil = m_pri_out*(h_RA-h_pri_out)
print('The cooling capacity of the auxiliary cooling coil is',Q_aux_coil,'Btu/hr')
print('The cooling capacity of the primary cooling coil is',Q_pri_coil,'Btu/hr')

The cooling capacity of the auxiliary cooling coil is 4201.555322012693 Btu/hr
The cooling capacity of the primary cooling coil is 7858.828131503438 Btu/hr
